# Save Lang2motion transformer model

In [ ]:
// for local development
%install-location /notebooks/language2motion.gt/swift-install
%install-swiftpm-flags -c release
%install '.package(path: "/notebooks/language2motion.gt")' Datasets TranslationModels TextModels ModelSupport SummaryWriter LangMotionModels Checkpoints

In [ ]:
import TensorFlow
import TextModels
import TranslationModels
import Foundation
import ModelSupport
import Datasets
import SummaryWriter
import LangMotionModels
import Checkpoints

## Set training params

In [ ]:
let maxTextSequenceLength =  20
let maxMotionLength =  100

In [ ]:
let dataURL = URL(fileURLWithPath: "/notebooks/language2motion.gt/data/")

## Instantiate model

In [ ]:
/// instantiate text processor
let vocabularyURL = dataURL.appendingPathComponent("vocab.txt")
let vocabulary: Vocabulary = try! Vocabulary(fromFile: vocabularyURL)
let tokenizer: Tokenizer = BERTTokenizer(vocabulary: vocabulary, caseSensitive: false, unknownToken: "[UNK]", maxTokenLength: nil)
let textProcessor = TextProcessor2(vocabulary: vocabulary, tokenizer: tokenizer, maxTextSequenceLength: maxTextSequenceLength, maxMotionLength: maxMotionLength)

/// instantiate model
let vocabSize = vocabulary.count
let nbJoints = 47 // TODO: get value from dataset
let nbMixtures = 20
let layerCount: Int = 6
let modelSize: Int = 256
let feedForwardSize: Int = 1024
let headCount: Int = 8
let dropoutProbability: Double = 0.1

var model = LangMotionTransformer(
    vocabSize: vocabSize, 
    nbJoints: nbJoints,
    nbMixtures: nbMixtures,
    layerCount: layerCount, 
    modelSize: modelSize, 
    feedForwardSize: feedForwardSize, 
    headCount: headCount, 
    dropoutProbability: dropoutProbability
)

## play with writer and reader

In [ ]:
let temporaryDirectory = dataURL.appendingPathComponent("CheckpointsTests", isDirectory: true)

In [ ]:
func setUp() {
    // Remove pre-existing test files.
    let headerPath = temporaryDirectory.appendingPathComponent(
        "testmodel.ckpt.index"
    ).path
    if FileManager.default.fileExists(atPath: headerPath) {
        try! FileManager.default.removeItem(atPath: headerPath)
    }
    let shardPath = temporaryDirectory.appendingPathComponent(
        "testmodel.ckpt.data-00000-of-00001"
    ).path
    if FileManager.default.fileExists(atPath: shardPath) {
        try! FileManager.default.removeItem(atPath: shardPath)
    }
}

In [ ]:
setUp()

In [ ]:
let vector = Tensor<Float>([1])
let matrix = Tensor<Float>([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
let ones = Tensor<Float>(ones: [1, 2, 2, 2, 2, 2, 1])
let tensor = Tensor<Float>(
    shape: [3, 4, 5], scalars: [Float](stride(from: 0.0, to: 60.0, by: 1.0)))

let tensors = [
    "model/vector": vector, "model/matrix": matrix, "ones": ones, "tensor": tensor,
]

do {
    let writer = CheckpointWriter(tensors: tensors)
    try writer.write(to: temporaryDirectory, name: "testmodel.ckpt")
} catch {
    print("Checkpoint writing / reading failed with error: \(error).")
}